# Federated Learning Training Plan: Create Plan

Let's try to make protobuf-serializable Training Plan and Model that work after deserializing :)

Current list of problems:
 * `tensor.shape` is not traceable inside the Plan (issue [#3554](https://github.com/OpenMined/PySyft/issues/3554)).
 * Autograd/Plan tracing doesn't work with native torch's loss functions and optimizers.
 * others?


In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings("ignore")

import syft as sy
import torch as th
from torch import jit
from torch import nn
from syft.serde import protobuf
import os
from syft.execution.state import State
from syft.execution.placeholder import PlaceHolder



sy.make_hook(globals())
# force protobuf serialization for tensors
hook.local_worker.framework = None
th.random.manual_seed(1)

W0527 14:55:18.152521 23204 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was 'D:\Anaconda3\envs\syft\lib\site-packages\tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0527 14:55:18.230508 23204 deprecation_wrapper.py:119] From D:\Anaconda3\envs\syft\lib\site-packages\tf_encrypted\session.py:24: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



Setting up Sandbox...
Done!


This utility function will serialize any object to protobuf binary and save to a file.

In [2]:
def serialize_to_bin_pb(worker, obj, filename):
    pb = protobuf.serde._bufferize(worker, obj)
    bin = pb.SerializeToString()
    print("Writing %s to %s/%s" % (obj.__class__.__name__, os.getcwd(), filename))
    with open(filename, "wb") as f:
        f.write(bin)


def set_model_params(module, params_list, start_param_idx=0):
    """ Set params list into model recursively
    """
    param_idx = start_param_idx

    for name, param in module._parameters.items():
        module._parameters[name] = params_list[param_idx]
        param_idx += 1

    for name, child in module._modules.items():
        if child is not None:
            param_idx += set_model_params(child, params_list, param_idx)

    return param_idx

## Step 1: Define the model

This model will train on MNIST data, it's very simple yet can demonstrate learning process.
There're 2 linear layers: 

* Linear 784x392
* ReLU
* Linear 392x10 

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 392)
        self.fc2 = nn.Linear(392, 10)

    def forward(self, x):
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.fc2(x)
        return x

model = Net()

## Step 2: Define Training Plan
### Loss function 
Batch size needs to be passed because otherwise `target.shape[0]` is not traced inside Plan yet (Issue [#3554](https://github.com/OpenMined/PySyft/issues/3554)).


In [4]:
def softmax_cross_entropy_with_logits(logits, targets, batch_size):
    """ Calculates softmax entropy
        Args:
            * logits: (NxC) outputs of dense layer
            * targets: (NxC) one-hot encoded labels
            * batch_size: value of N, temporarily required because Plan cannot trace .shape
    """
    # numstable logsoftmax
    norm_logits = logits - logits.max()
    log_probs = norm_logits - norm_logits.exp().sum(dim=1, keepdim=True).log()
    # NLL, reduction = mean
    return -(targets * log_probs).sum() / batch_size

### Optimization function
 
Just updates weights with grad*lr.

Note: can't do inplace update because of Autograd/Plan tracing specifics.

In [5]:
def naive_sgd(param, **kwargs):
    return param - kwargs['lr'] * param.grad

### Training Plan procedure

We define a routine that will take one batch of training data, and model parameters,
and will update model parameters to optimize them for given loss function using SGD.

In [6]:
@sy.func2plan()
def training_plan(X, y, batch_size, lr, model_params):
    # inject params into model
    set_model_params(model, model_params)

    # forward pass
    logits = model.forward(X)
    
    # loss
    loss = softmax_cross_entropy_with_logits(logits, y, batch_size)

    # backprop
    loss.backward()

    # step
    updated_params = [
        naive_sgd(param, lr=lr)
        for param in model_params
    ]
    
    # accuracy
    pred = th.argmax(logits, dim=1)
    target = th.argmax(y, dim=1)
    acc = pred.eq(target).sum().float() / batch_size

    return (
        loss,
        acc,
        *updated_params
    )

Let's build this procedure into the Plan that we can serialize.

In [7]:
# Dummy input parameters to make the trace
model_params = list(model.parameters())
X = th.randn(3, 28 * 28)
y = nn.functional.one_hot(th.tensor([1, 2, 3]), 10)
lr = th.tensor([0.01])
batch_size = th.tensor([3.0])

_ = training_plan.build(X, y, batch_size, lr, model_params, trace_autograd=True)

Let's look inside the Syft Plan and print out the list of operations recorded.

In [8]:
print(training_plan.code)

def training_plan(arg_1, arg_2, arg_3, arg_4, arg_5, arg_6, arg_7, arg_8):
    2 = arg_1.dim()
    var_0 = arg_5.t()
    var_1 = arg_1.matmul(var_0)
    var_2 = arg_6.add(var_1)
    var_3 = var_2.relu()
    2 = var_3.dim()
    var_4 = arg_7.t()
    var_5 = var_3.matmul(var_4)
    var_6 = arg_8.add(var_5)
    var_7 = var_6.max()
    var_8 = var_6.sub(var_7)
    var_9 = var_8.exp()
    var_10 = var_9.sum(dim=1, keepdim=True)
    var_11 = var_10.log()
    var_12 = var_8.sub(var_11)
    var_13 = arg_2.mul(var_12)
    var_14 = var_13.sum()
    var_15 = var_14.neg()
    out_1 = var_15.div(arg_3)
    var_16 = out_1.mul(0)
    var_17 = var_16.add(1)
    var_18 = var_17.div(arg_3)
    var_19 = var_18.mul(-1)
    var_20 = var_13.mul(0)
    var_21 = var_20.add(1)
    var_22 = var_21.mul(var_19)
    var_23 = var_22.mul(var_12)
    var_24 = var_22.mul(arg_2)
    var_25 = var_23.copy()
    var_26 = var_24.add(0)
    var_27 = var_24.mul(-1)
    var_28 = var_27.sum(dim=[1], keepdim=True)
    var_29 = 

Plan should be automatically translated to torchscript, too.
Let's examine torchscript code:

In [9]:
print(training_plan.torchscript.code)

def <Plan training_plan id:27645662168 owner:me built>
(argument_0: Tensor,
    argument_1: Tensor,
    argument_2: Tensor,
    argument_3: Tensor,
    argument_4: List[Tensor]) -> Tuple[Tensor, Tensor, Tensor, Tensor, Tensor, Tensor]:
  _0, _1, _2, _3, = argument_4
  _4 = torch.add(_1, torch.matmul(argument_0, torch.t(_0)), alpha=1)
  _5 = torch.relu(_4)
  _6 = torch.t(_2)
  _7 = torch.add(_3, torch.matmul(_5, _6), alpha=1)
  _8 = torch.sub(_7, torch.max(_7), alpha=1)
  _9 = torch.exp(_8)
  _10 = torch.sum(_9, [1], True, dtype=None)
  _11 = torch.sub(_8, torch.log(_10), alpha=1)
  _12 = torch.mul(argument_1, _11)
  _13 = torch.div(torch.neg(torch.sum(_12, dtype=None)), argument_2)
  _14 = torch.add(torch.mul(_13, CONSTANTS.c0), CONSTANTS.c1, alpha=1)
  _15 = torch.mul(torch.div(_14, argument_2), CONSTANTS.c2)
  _16 = torch.add(torch.mul(_12, CONSTANTS.c0), CONSTANTS.c1, alpha=1)
  _17 = torch.mul(torch.mul(_16, _15), argument_1)
  _18 = torch.add(_17, CONSTANTS.c0, alpha=1)
  _19 = to

## Step 3: Serialize!

Now it's time to serialize model params and plans to protobuf and save them for further usage:
 * In "Execute Plan" notebook, we load and execute these plans & model, from Python.
 * In "Host Plan" notebook, we send these plans & model to PyGrid, so it can be executed from other worker (e.g. syft.js).

**NOTE:**
 * We don't serialize full Model, only weights. How the Model is serialized is TBD.
   State is suitable protobuf class to wrap list of Model params tensors.

In [10]:
serialize_to_bin_pb(hook.local_worker, training_plan, "tp_full.pb")

# wrap weights in State to serialize
model_params_state = State(
    state_placeholders=[
        PlaceHolder().instantiate(param)
        for param in model_params
    ]
)

serialize_to_bin_pb(hook.local_worker, model_params_state, "model_params.pb")


Writing Plan to D:\projects\openmined\PySyft\examples\experimental\FL Training Plan/tp_full.pb
Writing State to D:\projects\openmined\PySyft\examples\experimental\FL Training Plan/model_params.pb
